For the resoultion of the 1st exercise I made these assumptions:

-The start and end times for the input time interval are provided as string inputs in the format 'YYYY-MM-DDTHH:MM:SS'.
-Faulty intervals refer to specific time ranges during which the metrics data is not reliable.
-We don't have any info related to the different kitchens.We don't have any kitchen column, or kitchen value. 
I don't want to assume that we don't have enough info to solve the problem (since we cannot filter for K1), so let's assume that the k1 is the combination of a1 and m1. 



In [37]:
import pandas as pd

def prepare_training_dataset(start_time, end_time):
    # Load the cooking metrics dataset
   # Load the datasets
    cooking_metrics = pd.read_csv('cooking_metrics.csv', delimiter=";", engine='python')
    batch_registry = pd.read_csv('batch_registry.csv', delimiter=";", engine='python')
    faulty_intervals = pd.read_csv('faulty_intervals.csv', delimiter=";", engine='python')

    # Filter data for machine m1 
    filtered_cooking_metrics = cooking_metrics[
        (cooking_metrics['machine_id'] == 'm1')
    ]

    # Filter data based on the specified time interval 
    filtered_cooking_metrics = filtered_cooking_metrics[
        (filtered_cooking_metrics['timestamp'] >= start_time) &
        (filtered_cooking_metrics['timestamp'] <= end_time)
    ]
    
    faulty_intervals = faulty_intervals[faulty_intervals['machine_id'] == 'm1']
    batch_registry = batch_registry[batch_registry['arepa_type'] == 'a1']

    # Remove faulty intervals that can be found in the corrispective csv
    for _, row in faulty_intervals.iterrows():
        faulty_start = row['start_time']
        faulty_end = row['end_time']
        filtered_cooking_metrics = filtered_cooking_metrics[
            ~((filtered_cooking_metrics['timestamp'] >= faulty_start) &
              (filtered_cooking_metrics['timestamp'] <= faulty_end))
        ]

    # Calculate hourly averaged metrics
    filtered_cooking_metrics['timestamp'] = pd.to_datetime(filtered_cooking_metrics['timestamp'])
    #hour_intervals = pd.date_range(start=start_date, end=end_date, freq='H')
    #hourly_averaged_metrics = filtered_cooking_metrics.resample('1H', on='timestamp').mean().reset_index()

    # Merge with batch registry to include batch information based on shared key
    training_dataset = pd.merge(filtered_cooking_metrics, batch_registry, on=['batch_id'])
    training_dataset['metric_1'] =  training_dataset['metric_1'].str.replace(',', '.').astype(float)
    training_dataset['metric_2'] =  training_dataset['metric_2'].str.replace(',', '.').astype(float)

    # Calculate the average between two metric values for the timestamp
    training_dataset['Average'] = (training_dataset['metric_1'] + training_dataset['metric_2']) / 2
    
    training_dataset.drop(['batch_id', 'metric_1','metric_2'], axis=1,inplace=True)
    
    return training_dataset

This code defines a function called prepare_training_dataset that takes the start and end times as input and returns the training dataset with hourly averaged metrics for the specified combination of machine m1 and arepa type a1 in a given time interval. 
The faulty intervals are filtered out before calculating the hourly averages, with the input data we have, there's no way to divide per hour.

To use the function, you can call it with the desired start and end times:

In [38]:
start_time = '2020-07-01T00:00:00'
end_time = '2023-07-02T00:00:00'
training_dataset = prepare_training_dataset(start_time, end_time)
print(training_dataset)

            timestamp machine_id arepa_type  Average
0 2020-11-01 01:30:00         m1         a1  0.47085


To scale the training to consider all machines in all kitchens associated with phase 1, but with a reduced list of arepa types provided in a text file, the code would need to be modified. 

Below are the required changes:

Load the list of arepa types from the text file: 
-Add an additional input parameter to the prepare_training_dataset function to accept the location of the text file containing the list of arepa types.
-Read the text file to extract the list of arepa types.

Modify the filtering step for arepa types:
-Instead of filtering based on a single arepa type, we have to use the list of arepa types obtained from the text file.
-Update the filtering condition to check if the arepa type is present in the list.
    